# AI Agent for Market Research & Competitive Analysis

This application serves as an interactive demo for a sophisticated AI research agent. Simply enter a company name and stock ticker to generate a multi-faceted analysis from three distinct investor perspectives.

## Step 1: API Key Setup

Before running this agent, you need two free API keys from Google. Follow these instructions carefully.

### 1. Get Your Google AI Studio API Key (`GOOGLE_API_KEY`)

1.  Go to **[Google AI Studio](https://aistudio.google.com/app/apikey)**.
2.  Click **"Create API key in new project"**.
3.  Copy the generated API key.

### 2. Get Your Google Programmable Search Engine Keys (`GOOGLE_CSE_ID`)

This is a two-part process to get a Search Engine ID and enable the API.

**Part A: Create the Search Engine**
1.  Go to the **[Programmable Search Engine control panel](https://programmablesearchengine.google.com/controlpanel/all)**.
2.  Click **"Add"** to create a new search engine.
3.  Name your search engine (e.g., "AI Agent Search").
4.  Crucially, select the option to **"Search the entire web"**.
5.  After it's created, go to the "Basics" tab and find the **"Search engine ID"**. Copy this ID.

**Part B: Enable the Custom Search API**
1.  Go to the **[Google Cloud Console API Library](https://console.cloud.google.com/apis/library/customsearch.googleapis.com)**.
2.  Ensure the project selected in the top navigation bar is the same one you created for your Google AI Studio key.
3.  Click the **"Enable"** button. If it's already enabled, you're all set.

### 3. Add Keys to Colab Secrets Manager

1.  In this notebook, click the **key icon (🔑)** in the left sidebar.
2.  Create a new secret named `GOOGLE_API_KEY` and paste your Google AI Studio key.
3.  Create another new secret named `GOOGLE_CSE_ID` and paste your Search Engine ID.

## Step 2: Install Dependencies & Setup Environment

This cell installs the required libraries, clones the project repository from GitHub to make the custom modules available, and sets up the necessary API keys for the agent to function.

In [ ]:
import importlib
import os
import sys
import requests
import re
from google.colab import userdata

def check_and_install_dependencies():
    requirements_url = 'https://raw.githubusercontent.com/eriktaylor/ai-agent-moat/main/requirements.txt'
    try:
        response = requests.get(requirements_url)
        response.raise_for_status()
        requirements = response.text.splitlines()

        import_name_map = {
            'faiss-cpu': 'faiss',
            'PyMuPDF': 'fitz',
            'google-api-python-client': 'googleapiclient',
            'beautifulsoup4': 'bs4',
            'sentence-transformers': 'sentence_transformers'
        }

        missing_packages = False
        for req in requirements:
            if req.strip() and not req.startswith('#'):
                package_name = re.split('[<>=~]=', req)[0].strip()
                import_name = import_name_map.get(package_name, package_name.replace('-', '_'))
                if not importlib.util.find_spec(import_name):
                    print(f"Missing dependency: {package_name} (checking for {import_name}) - will install all requirements.")
                    missing_packages = True
                    break
        
        if missing_packages:
            print("Installing dependencies from requirements.txt...")
            !pip install -q -r {requirements_url}
            print("Installation complete.")
        else:
            print("All dependencies are already installed.")

    except requests.RequestException as e:
        print(f"Error fetching requirements.txt: {e}")
        print("Proceeding with default installation...")
        !pip install -q -r {requirements_url}

check_and_install_dependencies()

repo_path = 'ai-agent-moat'
if not os.path.exists(repo_path):
    print(f"Cloning repository...")
    !git clone https://github.com/eriktaylor/ai-agent-moat.git
else:
    print("Repository already cloned.")

if repo_path not in sys.path:
    sys.path.append(repo_path)
    print(f"Added {repo_path} to system path.")

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_CSE_ID'] = userdata.get('GOOGLE_CSE_ID')

## Step 3: Import and Initialize the Agent

Now we import our custom-built agent and tools. The heavy lifting and complex logic are handled in the background by our `research_agent.py` and `tools.py` files.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings
from research_agent import ResearchAgent
from tools import get_stock_info

# Initialize models
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.2)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Instantiate the agent
research_agent = ResearchAgent(llm=llm, embeddings_model=embeddings)

print("AI Research Agent is initialized and ready.")

## Step 4: Run Your Analysis

Enter a company name and its corresponding stock ticker below to begin the analysis.

In [ ]:
from IPython.display import display, HTML, clear_output

company_name = input("Enter the company name (e.g., NVIDIA): ")
stock_ticker = input("Enter the stock ticker (e.g., NVDA): ")

clear_output(wait=True) # Clears the input prompts for a cleaner display

# To get fresh data and not use the cache, you can uncomment the next line:
# research_agent.clear_cache()

# Helper function to format and display sources
def display_analysis(title, result):
    print(f"\n--- {title} for {company_name} ---")
    answer = result.get('answer', 'No analysis generated.')
    sources = result.get('sources', [])
    
    # Display the main answer
    display(HTML(f"<div style='border: 1px solid #444; border-radius: 8px; padding: 20px; max-height: 500px; overflow-y: auto; white-space: pre-wrap; font-family: \"SF Pro Text\", \"Inter\", sans-serif; line-height: 1.6; background-color: #2c2c2e; color: #f0f0f0;'>{answer}</div>"))
    
    # Display the sources
    if sources:
        sources_html = "<div style='margin-top: 10px;'><strong>Sources Used:</strong><ul>"
        for source in sources:
            # Make sure title is a string before trying to escape it
            title_text = str(source.get('title', 'Untitled Source'))
            sources_html += f"<li><a href='{source.get('source', '#')}' target='_blank' style='color: #8ab4f8;'>{HTML(title_text).data}</a></li>"
        sources_html += "</ul></div>"
        display(HTML(sources_html))

# --- 1. KEY FINANCIAL DATA ---
print(f"--- 1. KEY FINANCIAL DATA for {stock_ticker.upper()} ---")
financial_data_raw = get_stock_info.run(stock_ticker) if stock_ticker else "No ticker provided."
display(HTML(f"<div style='border: 1px solid #444; border-radius: 8px; padding: 20px; white-space: pre-wrap; font-family: monospace; line-height: 1.6; background-color: #2c2c2e; color: #f0f0f0;'>{financial_data_raw}</div>"))

# --- 2. AI-GENERATED MARKET INVESTOR OUTLOOK ---
market_outlook_result = research_agent.generate_market_outlook(company_name, stock_ticker)
display_analysis("2. AI-GENERATED MARKET INVESTOR OUTLOOK", market_outlook_result)

# --- 3. AI-GENERATED VALUE INVESTOR ANALYSIS ---
value_analysis_result = research_agent.generate_value_analysis(company_name, stock_ticker)
display_analysis("3. AI-GENERATED VALUE INVESTOR ANALYSIS", value_analysis_result)

# --- 4. AI-GENERATED DEVIL'S ADVOCATE VIEW ---
devils_advocate_result = research_agent.generate_devils_advocate_view(company_name, stock_ticker)
display_analysis("4. AI-GENERATED DEVIL'S ADVOCATE VIEW", devils_advocate_result)